In [ ]:
from intelhex import IntelHex
ih = IntelHex()
hexFileName = "2bitTT.hex"

In [ ]:
#---AND Truth table
#--- 0 - 1 -------
#--0 0 - 0 -------
#--1 0 - 1 -------
#To Write another way;
# 00:0
# 01:0
# 10:0
# 11:1
bit2Truth ={ 0:0, 1:0, 2:0, 3:1};
bit2Truth[3]
ih.fromdict(bit2Truth)


In [74]:
from intelhex import IntelHex
import csv

class HexTruth:
    def __init__(self, csvFilename, hexFilename=None ):
        if hexFilename is None: hexFilename = csvFileName.split(".")[0]+ ".hex"
        self.csvFilename = csvFilename
        self.hexFilename = hexFilename
        self.ih= IntelHex()
        
    def writeFromIntelHex(self):
        f = open(self.hexFilename, 'w');
        self.ih.write_hex_file(f)
        f.close()
    
    def importCSV(self):
        self.csvTruth = []
        with open(self.csvFilename, newline='') as csvfile:
            csvReadingMachine = csv.reader(csvfile, delimiter=',', quotechar='"')
            for row in csvReadingMachine:
                self.csvTruth.append(row)
        return self.csvTruth
    
    def parseCSV(self, csvTruth, rangeIn = [0,8], rangeOut = [10]):
        self.hexDict = {};
        for line in csvTruth:
            if "#" in str(line):continue
            [addr, dat] = str(line).split("''")
            rangeIn = [0, addr.count(',') -1]
            if dat.count(",") is 1: rangeOut = [rangeIn[-1] + 2]
            else:rangeOut = [rangeIn[-1] + 2, rangeIn[-1] + 1 + dat.count(",")]
                       
            binaryString = ""
            for j in rangeIn:                
                binaryString+=str(line[j])
            address = (int(binaryString,2))
            binaryString = ""
            for j in rangeOut:
                binaryString+=str(line[j])
            data = (int(binaryString,2))
            self.hexDict[address] = data
        return self.hexDict

In [78]:
tt = HexTruth("b2TruthTable.csv")
cTruth = tt.importCSV()
print(cTruth)
newDict = tt.parseCSV(cTruth)
tt.ih.fromdict(newDict)
tt.writeFromIntelHex()

[['0', '0', '', '0'], ['0', '1', '', '0'], ['1', '0', '', '0'], ['1', '1', '', '1']]


/home/jake/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:13: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.


In [77]:
for j in [4,4]:
    print(j)

4
4


In [17]:
st ="ho.hi"
st.split(".")

['ho', 'hi']

In [40]:
len(hope)

4

In [2]:
csvFileName = 'b2TruthTable.csv'


        

In [4]:
from intelhex import IntelHex
ih = IntelHex()
csvList = importCSV("b2TruthTable.csv")
hexDict = parseCSV(csvList, rangeIn = [0,1], rangeOut = [3])
ih.fromdict(hexDict)
ih.dump()
writeFromIntelHex(ih)

0000  00 00 00 01 -- -- -- -- -- -- -- -- -- -- -- --  |....            |


/home/jake/anaconda3/lib/python3.4/site-packages/ipykernel/__main__.py:3: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  app.launch_new_instance()


In [11]:
ih.dump()


0000  00 00 00 01 -- -- -- -- -- -- -- -- -- -- -- --  |....            |


In [ ]:
synthesizeHex()

In [ ]:
hex(110)

In [ ]:
ih.dump()


In [ ]:
hex(int(("0110"),2))